In [1]:
# LazyJSON would be faster, but having issues installing to 0.6.1
using Query, DataFrames, CSV, JSON

INFO: Recompiling stale cache file /home/oleg/Localdev/anaconda/envs/julia/share/julia/site/lib/v0.6/TableTraits.ji for module TableTraits.
INFO: Recompiling stale cache file /home/oleg/Localdev/anaconda/envs/julia/share/julia/site/lib/v0.6/IterableTables.ji for module IterableTables.
INFO: Recompiling stale cache file /home/oleg/Localdev/anaconda/envs/julia/share/julia/site/lib/v0.6/MacroTools.ji for module MacroTools.
INFO: Recompiling stale cache file /home/oleg/Localdev/anaconda/envs/julia/share/julia/site/lib/v0.6/QueryOperators.ji for module QueryOperators.
INFO: Recompiling stale cache file /home/oleg/Localdev/anaconda/envs/julia/share/julia/site/lib/v0.6/DataFrames.ji for module DataFrames.
INFO: Recompiling stale cache file /home/oleg/Localdev/anaconda/envs/julia/share/julia/site/lib/v0.6/CSV.ji for module CSV.


Import list of stations, departures, arrivals

In [2]:
haltestellen = CSV.read("data/haltestellen-in-metro.csv")
head(haltestellen)

,xtf_id,Nummer,Name,Abkuerzung,DatenherrA,TUNummer,TUAbkuerzu,Betriebspu,Verkehrsmi,Hoehe,GdeNummer,GdeName,BeginnGuel,EndeGuelti,Bearbeitun,Stand,rUebergeor
1,ch14uvag00047298,8503391,"Goldingen, Vordersagen",missing,missing,7,PAG,Haltestelle,Bus,780,3342,Eschenbach (SG),2014/12/01,missing,2017/12/01,2017/12/10,missing
2,ch14uvag00058578,8503479,"Schaffhausen, Kreuz",missing,missing,165,RVSH,Haltestelle,Bus,385,2939,Schaffhausen,2014/12/01,missing,2017/12/01,2017/12/10,missing
3,ch14uvag00042993,8591837,"Cham, Lorzenpark",missing,missing,185,ZVB,Haltestelle,Bus,432,1702,Cham,2014/12/01,missing,2017/12/01,2017/12/10,missing
4,ch14uvag00061208,8573492,"Thayngen, Bahnhof",missing,missing,165,RVSH,Haltestelle,Bus,417,2920,Thayngen,2014/12/01,missing,2017/12/01,2017/12/10,missing
5,ch14uvag00060992,8582713,"Tägerig, Weidhof",missing,missing,7,PAG,Haltestelle,Bus,378,4077,Tägerig,2014/12/01,missing,2017/12/01,2017/12/10,missing
6,ch14uvag00063496,8592431,"Weggis, Post",missing,missing,340,AAGS,Haltestelle,Bus,445,1069,Weggis,2014/12/01,missing,2017/12/01,2017/12/10,missing


In [5]:
open("data/Metroregion.geojson", "r") do f
    global metroregion
    metroregion = JSON.parse(readstring(f))
end

Dict{String,Any} with 4 entries:
  "name"     => "Metroregion"
  "features" => Any[Dict{String,Any}(Pair{String,Any}("properties", Dict{String…
  "type"     => "FeatureCollection"
  "crs"      => Dict{String,Any}(Pair{String,Any}("properties", Dict{String,Any…

In [7]:
metronumbers = [ i["properties"]["BFS_NUMMER"] for i in metroregion["features"] ]
length(metronumbers)

343

Run a query to filter to the MKZ communes.

In [41]:
ZURICH_HB = "8503000"
ZURICH_SZU = "8503088"
q1 = @from i in haltestellen begin
    @where i.GdeNummer in metronumbers && i.Nummer != ZURICH_HB && i.Nummer != ZURICH_SZU
    @select {id=i.Nummer, name=i.Name, geoid=i.GdeNummer, geoname=i.GdeName}
    @collect DataFrame
end

,id,name,geoid,geoname
1,8503391,"Goldingen, Vordersagen",3342,Eschenbach (SG)
2,8503479,"Schaffhausen, Kreuz",2939,Schaffhausen
3,8591837,"Cham, Lorzenpark",1702,Cham
4,8573492,"Thayngen, Bahnhof",2920,Thayngen
5,8582713,"Tägerig, Weidhof",4077,Tägerig
6,8592431,"Weggis, Post",1069,Weggis
7,8580276,"Wohlen AG, Luegisland",4082,Wohlen (AG)
8,8576957,"Tänikon, Forschungsanstalt",4551,Aadorf
9,8590957,"Winterthur, Hedy Hahnloser",230,Winterthur
10,8506004,Kollbrunn,231,Zell (ZH)


search: else elseif ifelse @elapsed LevelsException fieldoffset



Load the current travel plan in [GTFS format](https://opentransportdata.swiss/de/cookbook/gtfs/#Wann_gibt_es_neue_GTFS_Static_Files)

In [14]:
stoptimes = CSV.read("data/gtfsfp/stop_times.txt")
head(stoptimes)

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type
1,1.TA.1-1-A-j18-1.1.H,01:00:00,01:00:00,8503000:0:41/42,1,0,0
2,1.TA.1-1-A-j18-1.1.H,01:02:00,01:02:00,8503020:0:4,2,0,0
3,1.TA.1-1-A-j18-1.1.H,01:05:00,01:05:00,8503001:0:3,3,0,0
4,1.TA.1-1-A-j18-1.1.H,01:07:00,01:07:00,8503509:0:3,4,0,0
5,1.TA.1-1-A-j18-1.1.H,01:10:00,01:10:00,8503512:0:1,5,0,0
6,1.TA.1-1-A-j18-1.1.H,01:13:00,01:15:00,8503508:0:3,6,0,0


In [34]:
stoptimes[:stop_id] = [ split(i[:stop_id],":")[1] for i in eachrow(stoptimes) ]
head(stoptimes)

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type
1,1.TA.1-1-A-j18-1.1.H,01:00:00,01:00:00,8503000,1,0,0
2,1.TA.1-1-A-j18-1.1.H,01:02:00,01:02:00,8503020,2,0,0
3,1.TA.1-1-A-j18-1.1.H,01:05:00,01:05:00,8503001,3,0,0
4,1.TA.1-1-A-j18-1.1.H,01:07:00,01:07:00,8503509,4,0,0
5,1.TA.1-1-A-j18-1.1.H,01:10:00,01:10:00,8503512,5,0,0
6,1.TA.1-1-A-j18-1.1.H,01:13:00,01:15:00,8503508,6,0,0


Calculate time difference

In [101]:
cur_trip_id = ""
cur_trip_start = 0
diff_times = Dict()

function get_23_time(the_time)
    bad_hour = parse(Int64, the_time[1:2])
    if bad_hour > 23
        the_time = replace(the_time, r"..(:..:..)", s"\1")
        the_time = string(bad_hour-24) * the_time
    end
    DateTime(the_time, dateformat"HH:MM:SS")
end

for i in eachrow(stoptimes)
    if cur_trip_id != i[:trip_id]
        cur_trip_id = ""
    end
    cur_stop_id = i[:stop_id]
    if cur_stop_id == ZURICH_HB
        cur_trip_id = i[:trip_id]
        cur_trip_start = get_23_time(i[:departure_time])
    elseif cur_trip_id != ""
        cur_trip_end = get_23_time(i[:arrival_time])
        if cur_trip_start != cur_trip_end && cur_trip_end != 0 && cur_trip_start != 0
            trip_time_diff = cur_trip_end - cur_trip_start
            #if haskey(diff_times, cur_stop_id)
            #    diff_times[cur_stop_id] = (diff_times[cur_stop_id] + trip_time_diff) / 2.0
            #else
            diff_times[cur_stop_id] = trip_time_diff
        end
    end
end
diff_times

Dict{Any,Any} with 261 entries:
  "8500309" => 2460000 milliseconds
  "8500218" => 2340000 milliseconds
  "8500026" => 2760000 milliseconds
  "8500302" => 3540000 milliseconds
  "8505305" => -77100000 milliseconds
  "8502229" => 600000 milliseconds
  "8500329" => 2640000 milliseconds
  "8503204" => 1260000 milliseconds
  "8503311" => 720000 milliseconds
  "8515995" => 2040000 milliseconds
  "8506109" => -82680000 milliseconds
  "8503104" => 1500000 milliseconds
  "8502200" => 1980000 milliseconds
  "8503207" => 1920000 milliseconds
  "8503512" => 660000 milliseconds
  "8503020" => 120000 milliseconds
  "8502114" => 3300000 milliseconds
  "8503224" => 3600000 milliseconds
  "8503226" => 2940000 milliseconds
  "8503122" => 1860000 milliseconds
  "8505007" => 4800000 milliseconds
  "8509003" => -81480000 milliseconds
  "8506016" => 1740000 milliseconds
  "8503236" => 4980000 milliseconds
  "8505201" => 8700000 milliseconds
  ⋮         => ⋮

Statistical summaries (max, min, average)

Visualisations

Export